Scratch code to create Rosette class with cadquery for stable boolean opereations

In [2]:
import cadquery as cq
import numpy as np

In [9]:
# geometric parameters
a, c, n_arms = 1, 4, 10
r0 = a/2
hp = a*1.5
h0 = r0/2

# create sphere
sphere = cq.Workplane().sphere(r0)

# function to create bullet, given normal vector and translation 
def create_bullet(a, c, hp, r0, h0, workplane):
    # create pyramid
    n_pyr = 6
    ri = a*np.cos(np.radians(30)) # distance between center and edge of hexagon
    theta = 90 - np.degrees(np.arctan(hp/ri))
    pyramid = workplane.polygon(n_pyr, 2*a).extrude(-hp, taper=theta)
    # create cylinder 
    n_cyl = 6
    cylinder = workplane.polygon(n_cyl, 2*a).extrude(2*c)
    # create bullet (union)
    bullet = cylinder.union(pyramid)
    return bullet

# create outer shell to "place" bullets on
# temp use fibbonaci for all values of n_arms
r_outer = r0 + hp - h0
# Modified fibbonaci lattice 
# Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
epsilon = 0.33
goldenRatio = (1 + 5**0.5)/2
i = np.arange(0, n_arms) 
theta = 2 * np.pi * i / goldenRatio
phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
outer_coords = r_outer*(np.column_stack((x, y, z)))

bullets = []
for i in range(len(outer_coords)):
    normal_vector = tuple(outer_coords[i])
    plane = cq.Plane(origin=normal_vector, normal=normal_vector)
    workplane = cq.Workplane(plane)
    bullet = create_bullet(a, c, hp, r0, h0, workplane)
    bullets.append(bullet)

# boolean union to create rosette
ros = sphere.union(bullets[0])
for i in range(1, n_arms):
    ros = ros.union(bullets[i])
ros

In [8]:
surface_area = ros.val().Area()
print(f"Surface area of the ros object: {surface_area}")

Surface area of the ros object: 669.3237074237784


In [ ]:
# create pyramid and move up
# create pyramid
n_pyr = 6
ri = a*np.cos(np.radians(30)) # distance between center and edge of hexagon
theta = 90 - np.degrees(np.arctan(hp/ri))
print(theta)
pyramid = cq.Workplane().polygon(n_pyr, 2*a).extrude(-hp, taper=theta)
# Get the bounding box of the pyramid
bbox = pyramid.val().BoundingBox()

# Calculate the height in the z direction
height_z = bbox.zmax - bbox.zmin
print(f"Height of the pyramid in the z direction: {height_z}")

pyramid = pyramid.translate((0,0,hp))
pyramid = pyramid.translate((0,0,r0))

In [ ]:
temp = pyramid.union(sphere)
temp

In [ ]:
center = sphere.val().Center()
print(center)

In [ ]:
volume = sphere.val().Volume()
print(f"Volume of the sphere: {volume}")

In [6]:
# # create bullet 

# # create pyramid
# n_pyr = 6
# theta = 90 - np.degrees(np.arctan(hp/a))
# pyramid = cq.Workplane().polygon(n_pyr, 2*a).extrude(-hp, taper=theta)

# # create cylinder 
# n_cyl = 6
# cylinder = cq.Workplane().polygon(n_cyl, 2*a).extrude(2*c)

# # create bullet (union)
# bullet = cylinder.union(pyramid)

# # shift bullet up so tip is at z=0
# bullet = bullet.translate((0,0,hp))

# bullet

In [ ]:
# create pyramid
n_pyr = 6
theta = 90 - np.degrees(np.arctan(hp/a))
print(theta)
pyramid = cq.Workplane().polygon(n_pyr, 2*a).extrude(hp, taper=theta)

In [ ]:
pyramid

In [ ]:
# Get the bounding box of the pyramid
bbox = pyramid.val().BoundingBox()

# Calculate the height in the z direction
height_z = bbox.zmax - bbox.zmin
print(f"Height of the pyramid in the z direction: {height_z}")

In [ ]:
len(outer_coords)

In [ ]:
# create outer shell to "place" bullets on
# temp use fibbonaci for all values of n_arms
r_outer = hp/2 + c - h0 + r0
# Modified fibbonaci lattice 
# Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
epsilon = 0.33
goldenRatio = (1 + 5**0.5)/2
i = np.arange(0, n_arms) 
theta = 2 * np.pi * i / goldenRatio
phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
outer_coords = r_outer*(np.column_stack((x, y, z)))

outer_coords

In [ ]:
# Translate bullet to new positions based on outer_coords
translated_bullets = [bullet.translate(coord) for coord in outer_coords]

# Combine all translated bullets into a single object
combined_bullets = translated_bullets[0]
for tb in translated_bullets[1:]:
    combined_bullets = combined_bullets.union(tb)

combined_bullets

In [ ]:
translated_bullet = bullet.translate((0, 0, r0-h0))
temp = sphere.union(translated_bullet)
temp

In [ ]:
# Create a unit sphere
unit_sphere = cq.Workplane("front").sphere(1)



In [ ]:
normal_vector = (1, 1, 1)
plane = cq.Plane(origin=cq.Vector(0, 0, 0), normal=cq.Vector(*normal_vector))  # Create a plane using the normal vector
workplane = cq.Workplane(plane) 
test = workplane.polygon(n_cyl, 1).extrude(4).translate((0,0,1))
test

In [ ]:
def orthogonal_workplane(v, r):
    # Normalize the vector
    v = v / np.linalg.norm(v)
    
    # Calculate the tangent point on the sphere
    tangent_point = r * v
    
    # Create the workplane at the tangent point and orthogonal to the vector
    workplane = cq.Workplane().transformed(offset=tangent_point, rotate=(0.0, 0.0, 0.0))
    
    return workplane

# Example usage
v = np.array([1, 1, 1])
r = 2
workplane = orthogonal_workplane(v, r)
workplane